# Main ML code below

In [1]:
import numpy as np

from IPython.display import Image
from IPython.display import Markdown

import matplotlib
import matplotlib.pyplot as plt

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import torch.optim as optim

from torch.autograd import Variable

In [2]:
# License: BSD
# Author: Sasank Chilamkurthy

# material from notebook at: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../../../stanford_dogs_new/'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
        
# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))
# 
# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)
# 
# imshow(out, title=[class_names[x] for x in classes])

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])
                
                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [3]:
vgg19 = models.vgg19(pretrained=True)

for param in vgg19.parameters():
    param.requires_grad = False

print(vgg19)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /home/rka73/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth



VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), pad

In [4]:
# ref: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

vgg19.classifier[6] = nn.Linear(in_features=4096, out_features=120, bias=True)

vgg19 = vgg19.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_vgg19 = optim.SGD(vgg19.parameters(), lr=0.001, momentum=0.9)
# optimizer_vgg19 = optim.Adamax(vgg19.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_vgg19, step_size=7, gamma=0.1)

In [5]:
vgg19_30 = train_model(vgg19, criterion, optimizer_vgg19, exp_lr_scheduler, num_epochs=30)

# 87.5244%


Epoch 0/29
----------
train Loss: 1.7397 Acc: 0.5753
val Loss: 0.6839 Acc: 0.8177

Epoch 1/29
----------
train Loss: 1.6007 Acc: 0.6391
val Loss: 0.7283 Acc: 0.8304

Epoch 2/29
----------
train Loss: 1.6333 Acc: 0.6468
val Loss: 0.7320 Acc: 0.8367

Epoch 3/29
----------
train Loss: 1.6756 Acc: 0.6530
val Loss: 0.7283 Acc: 0.8328

Epoch 4/29
----------
train Loss: 1.6828 Acc: 0.6606
val Loss: 0.7281 Acc: 0.8333

Epoch 5/29
----------
train Loss: 1.6917 Acc: 0.6652
val Loss: 0.7278 Acc: 0.8367

Epoch 6/29
----------
train Loss: 1.6722 Acc: 0.6751
val Loss: 0.8493 Acc: 0.8275

Epoch 7/29
----------
train Loss: 1.4456 Acc: 0.7012
val Loss: 0.6094 Acc: 0.8645

Epoch 8/29
----------
train Loss: 1.4115 Acc: 0.7054
val Loss: 0.5978 Acc: 0.8655

Epoch 9/29
----------
train Loss: 1.3650 Acc: 0.7055
val Loss: 0.5858 Acc: 0.8665

Epoch 10/29
----------
train Loss: 1.3497 Acc: 0.7075
val Loss: 0.5779 Acc: 0.8674

Epoch 11/29
----------
train Loss: 1.3243 Acc: 0.7100
val Loss: 0.5603 Acc: 0.8660

Ep

In [6]:
torch.save(vgg19_30.state_dict(), 'vgg19_based_model2.pt')

## Load the Model and test it

In [12]:
vgg19_loaded = models.vgg19(pretrained=True)

for param in vgg19_loaded.parameters():
    param.requires_grad = False

vgg19_loaded.classifier = nn.Linear(in_features=4096, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg19_loaded = vgg19_loaded.to(device)



vgg19_loaded.load_state_dict(torch.load('vgg19_based_model2.pt'), strict=False)
vgg19_loaded.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

## train vgg19 with Adamax optimizer

In [13]:
# ref: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

vgg19.classifier[6] = nn.Linear(in_features=4096, out_features=120, bias=True)

vgg19 = vgg19.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
# optimizer_vgg19 = optim.SGD(vgg19.parameters(), lr=0.001, momentum=0.9)
optimizer_vgg19_adamax = optim.Adamax(vgg19.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_vgg19_adamax, step_size=7, gamma=0.1)

In [14]:
vgg19_adamax_30ep = train_model(vgg19, criterion, optimizer_vgg19_adamax, exp_lr_scheduler, num_epochs=30)

# 87.6706%


Epoch 0/29
----------
train Loss: 1.4576 Acc: 0.6109
val Loss: 0.4763 Acc: 0.8523

Epoch 1/29
----------
train Loss: 1.0848 Acc: 0.6977
val Loss: 0.4571 Acc: 0.8548

Epoch 2/29
----------
train Loss: 1.0810 Acc: 0.7072
val Loss: 0.4519 Acc: 0.8596

Epoch 3/29
----------
train Loss: 1.0646 Acc: 0.7167
val Loss: 0.4600 Acc: 0.8596

Epoch 4/29
----------
train Loss: 1.0551 Acc: 0.7234
val Loss: 0.4452 Acc: 0.8713

Epoch 5/29
----------
train Loss: 1.0536 Acc: 0.7277
val Loss: 0.4464 Acc: 0.8670

Epoch 6/29
----------
train Loss: 1.0706 Acc: 0.7265
val Loss: 0.4628 Acc: 0.8631

Epoch 7/29
----------
train Loss: 1.0332 Acc: 0.7456
val Loss: 0.4432 Acc: 0.8694

Epoch 8/29
----------
train Loss: 1.0211 Acc: 0.7418
val Loss: 0.4404 Acc: 0.8718

Epoch 9/29
----------
train Loss: 0.9901 Acc: 0.7499
val Loss: 0.4371 Acc: 0.8709

Epoch 10/29
----------
train Loss: 1.0093 Acc: 0.7437
val Loss: 0.4381 Acc: 0.8718

Epoch 11/29
----------
train Loss: 1.0021 Acc: 0.7459
val Loss: 0.4355 Acc: 0.8748

Ep

In [15]:
torch.save(vgg19_adamax_30ep.state_dict(), 'vgg19_adamax_30ep_based_model2.pt')

## Load the Adamax optimizer based vgg19 model and test it

In [17]:
vgg19_adamax_30ep_loaded = models.vgg19(pretrained=True)

for param in vgg19_adamax_30ep_loaded.parameters():
    param.requires_grad = False

vgg19_adamax_30ep_loaded.classifier = nn.Linear(in_features=4096, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg19_adamax_30ep_loaded = vgg19_adamax_30ep_loaded.to(device)



vgg19_adamax_30ep_loaded.load_state_dict(torch.load('vgg19_adamax_30ep_based_model2.pt'), strict=False)
vgg19_adamax_30ep_loaded.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd